# Adder

Adding is quite straightforward, with the necessary circuits provides by qiskit

In [19]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.circuit.library import VBERippleCarryAdder

# we do an example of 5 + 3 in a 4-bit registers

bitLength = 4

firstRegister = QuantumRegister(bitLength, 'first')
secondRegister = QuantumRegister(bitLength, 'second')
carryRegister = QuantumRegister(1, 'carry')
ancillaRegister = QuantumRegister(bitLength, 'ancilla')

secondClassicalRegister = ClassicalRegister(bitLength-1, 'second_classical')


adder = VBERippleCarryAdder(bitLength, name="Adder")
num_qubits = len(adder.qubits)

print(num_qubits)
# define number loader
def loadNumber(circ: QuantumCircuit, register: QuantumRegister, number: int):
    number_in_binary = '0'*(register.size - len(bin(number)[2:])) + bin(number)[2:]
    
    for i in range(len(number_in_binary)):
        if number_in_binary[::-1][i] == '1':
            circ.x(register[i])

firstNumber = 5
secondNumber = 3
circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, secondClassicalRegister)

# load first number
loadNumber(circ, firstRegister, firstNumber)
# load second number
loadNumber(circ, secondRegister, secondNumber)

circ.barrier()

circ.append(adder, list(range(num_qubits)))
circ.barrier()
# circ.measure(firstRegister, firstClassicalRegister)
circ.measure(secondRegister[:-1], secondClassicalRegister)
# circ.measure(ancillaRegister, ancillaClassicalRegister)
circ.draw()


13


░ ┌─────────┐ ░          
             carry: ──────░─┤0        ├─░──────────
                    ┌───┐ ░ │         │ ░          
           first_0: ┤ X ├─░─┤1        ├─░──────────
                    └───┘ ░ │         │ ░          
           first_1: ──────░─┤2        ├─░──────────
                    ┌───┐ ░ │         │ ░          
           first_2: ┤ X ├─░─┤3        ├─░──────────
                    └───┘ ░ │         │ ░          
           first_3: ──────░─┤4        ├─░──────────
                    ┌───┐ ░ │         │ ░ ┌─┐      
          second_0: ┤ X ├─░─┤5        ├─░─┤M├──────
                    ├───┤ ░ │         │ ░ └╥┘┌─┐   
          second_1: ┤ X ├─░─┤6  Adder ├─░──╫─┤M├───
                    └───┘ ░ │         │ ░  ║ └╥┘┌─┐
          second_2: ──────░─┤7        ├─░──╫──╫─┤M├
                          ░ │         │ ░  ║  ║ └╥┘
          second_3: ──────░─┤8        ├─░──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║ 
         ancilla_0: ──────░─┤9        ├─░──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║ 
         ancilla_1: ──────░─┤10       ├─░──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║ 
         ancilla_2: ──────░─┤11       ├─░──╫──╫──╫─
                          ░ │         │ ░  ║  ║  ║ 
         ancilla_3: ──────░─┤12       ├─░──╫──╫──╫─
                          ░ └─────────┘ ░  ║  ║  ║ 
second_classical: 3/═══════════════════════╩══╩══╩═
                                           0  1  2

In [20]:
job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
print(counts)

{'000': 1000}


In [4]:
def resultToNumber(counts: dict):
    results_dict = {}
    for keys, value in counts.items():
        splitted_measurements = keys.split(' ')
        # output = int(splitted_measurements[1], 2)
        output = splitted_measurements[1]
        results_dict[output] = value
    return results_dict

for firstNumber in range(6):
    for secondNumber in range(6):
        circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
        # load first number
        loadNumber(circ, firstRegister, firstNumber)
        # load second number
        loadNumber(circ, secondRegister, secondNumber)
        circ.barrier()
        circ.append(adder, list(range(num_qubits)))
        circ.barrier()
        circ.measure(firstRegister, firstClassicalRegister)
        circ.measure(secondRegister, secondClassicalRegister)
        circ.measure(ancillaRegister, ancillaClassicalRegister)

        job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
        counts = job.result().get_counts()
        processed_counts = resultToNumber(counts)
        print("{} + {} ".format(firstNumber, secondNumber), processed_counts)
        


0 + 0  {'0000': 1000}
0 + 1  {'0001': 1000}
0 + 2  {'0010': 1000}
0 + 3  {'0011': 1000}
0 + 4  {'0100': 1000}
0 + 5  {'0101': 1000}
1 + 0  {'0001': 1000}
1 + 1  {'0010': 1000}
1 + 2  {'0011': 1000}
1 + 3  {'0100': 1000}
1 + 4  {'0101': 1000}
1 + 5  {'0110': 1000}
2 + 0  {'0010': 1000}
2 + 1  {'0011': 1000}
2 + 2  {'0100': 1000}
2 + 3  {'0101': 1000}
2 + 4  {'0110': 1000}
2 + 5  {'0111': 1000}
3 + 0  {'0011': 1000}
3 + 1  {'0100': 1000}
3 + 2  {'0101': 1000}
3 + 3  {'0110': 1000}
3 + 4  {'0111': 1000}
3 + 5  {'1000': 1000}
4 + 0  {'0100': 1000}
4 + 1  {'0101': 1000}
4 + 2  {'0110': 1000}
4 + 3  {'0111': 1000}
4 + 4  {'1000': 1000}
4 + 5  {'1001': 1000}
5 + 0  {'0101': 1000}
5 + 1  {'0110': 1000}
5 + 2  {'0111': 1000}
5 + 3  {'1000': 1000}
5 + 4  {'1001': 1000}
5 + 5  {'1010': 1000}


# subtractor

Subtraction can work by 
$$a-b = a + \neg ||b|| +1$$

key thing to take note here is the range of values

In [28]:
firstClassicalRegister = ClassicalRegister(bitLength, 'first_classical')
secondClassicalRegister = ClassicalRegister(bitLength, 'second_classical')
ancillaClassicalRegister = ClassicalRegister(bitLength, 'ancilla_classical')

def resultToNumber(counts: dict):
    results_dict = {}
    for keys, value in counts.items():
        splitted_measurements = keys.split(' ')
        # output = int(splitted_measurements[1], 2)
        output = splitted_measurements[1]
        results_dict[output] = value
    return results_dict

def process_results(counts: dict):
    num = list(counts.keys())[0]
    if num[0] == '0':
        return int(num, 2)
    else:
        return int(num[1:], 2) - 2**len(num[1:])

for firstNumber in range(2**(bitLength-1)): # the first digit is the sign, so does not encode the value
    for secondNumber in range(2**(bitLength-1)):
        circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
        # load first number
        loadNumber(circ, firstRegister, firstNumber)
        # load second number
        loadNumber(circ, secondRegister, secondNumber)
        circ.barrier()
        circ.x(secondRegister)
        circ.x(carryRegister)
        circ.append(adder, list(range(num_qubits)))
        circ.barrier()
        circ.measure(firstRegister, firstClassicalRegister)
        circ.measure(secondRegister, secondClassicalRegister)
        circ.measure(ancillaRegister, ancillaClassicalRegister)

        job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
        counts = job.result().get_counts()
        processed_counts = resultToNumber(counts)
        processed_results = process_results(processed_counts)
        print("{} - {} ".format(firstNumber, secondNumber), processed_counts, processed_results)

0 - 0  {'0000': 1000} 0
0 - 1  {'1111': 1000} -1
0 - 2  {'1110': 1000} -2
0 - 3  {'1101': 1000} -3
0 - 4  {'1100': 1000} -4
0 - 5  {'1011': 1000} -5
0 - 6  {'1010': 1000} -6
0 - 7  {'1001': 1000} -7
1 - 0  {'0001': 1000} 1
1 - 1  {'0000': 1000} 0
1 - 2  {'1111': 1000} -1
1 - 3  {'1110': 1000} -2
1 - 4  {'1101': 1000} -3
1 - 5  {'1100': 1000} -4
1 - 6  {'1011': 1000} -5
1 - 7  {'1010': 1000} -6
2 - 0  {'0010': 1000} 2
2 - 1  {'0001': 1000} 1
2 - 2  {'0000': 1000} 0
2 - 3  {'1111': 1000} -1
2 - 4  {'1110': 1000} -2
2 - 5  {'1101': 1000} -3
2 - 6  {'1100': 1000} -4
2 - 7  {'1011': 1000} -5
3 - 0  {'0011': 1000} 3
3 - 1  {'0010': 1000} 2
3 - 2  {'0001': 1000} 1
3 - 3  {'0000': 1000} 0
3 - 4  {'1111': 1000} -1
3 - 5  {'1110': 1000} -2
3 - 6  {'1101': 1000} -3
3 - 7  {'1100': 1000} -4
4 - 0  {'0100': 1000} 4
4 - 1  {'0011': 1000} 3
4 - 2  {'0010': 1000} 2
4 - 3  {'0001': 1000} 1
4 - 4  {'0000': 1000} 0
4 - 5  {'1111': 1000} -1
4 - 6  {'1110': 1000} -2
4 - 7  {'1101': 1000} -3
5 - 0  {'0101':

In [6]:
firstNumber = 2
secondNumber = 6
circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
# load first number
loadNumber(circ, firstRegister, firstNumber)
# load second number
loadNumber(circ, secondRegister, secondNumber)
circ.barrier()
circ.x(secondRegister)
circ.x(carryRegister)
circ.append(adder, list(range(num_qubits)))
circ.barrier()
circ.measure(firstRegister, firstClassicalRegister)
circ.measure(secondRegister, secondClassicalRegister)
circ.measure(ancillaRegister, ancillaClassicalRegister)

job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
processed_counts = resultToNumber(counts)
print("{} - {} ".format(firstNumber, secondNumber), processed_counts)

2 - 6  {'1100': 1000}


# Test Adding constant

In [7]:
firstNumber = 2
secondNumber = 6
constant = 4
circ = QuantumCircuit(carryRegister, firstRegister, secondRegister, ancillaRegister, firstClassicalRegister, secondClassicalRegister, ancillaClassicalRegister)
# load first number
loadNumber(circ, firstRegister, firstNumber)
# load second number
loadNumber(circ, secondRegister, secondNumber)
circ.barrier()
circ.x(secondRegister)
circ.x(carryRegister)
circ.append(adder, list(range(num_qubits)))
circ.barrier()

circ.measure(firstRegister, firstClassicalRegister)
circ.measure(secondRegister, secondClassicalRegister)
circ.measure(ancillaRegister, ancillaClassicalRegister)

circ.draw()


░ ┌───┐┌─────────┐ ░                               »
              carry: ──────░─┤ X ├┤0        ├─░───────────────────────────────»
                           ░ └───┘│         │ ░ ┌─┐                           »
            first_0: ──────░──────┤1        ├─░─┤M├───────────────────────────»
                     ┌───┐ ░      │         │ ░ └╥┘┌─┐                        »
            first_1: ┤ X ├─░──────┤2        ├─░──╫─┤M├────────────────────────»
                     └───┘ ░      │         │ ░  ║ └╥┘┌─┐                     »
            first_2: ──────░──────┤3        ├─░──╫──╫─┤M├─────────────────────»
                           ░      │         │ ░  ║  ║ └╥┘┌─┐                  »
            first_3: ──────░──────┤4        ├─░──╫──╫──╫─┤M├──────────────────»
                           ░ ┌───┐│         │ ░  ║  ║  ║ └╥┘┌─┐               »
           second_0: ──────░─┤ X ├┤5        ├─░──╫──╫──╫──╫─┤M├───────────────»
                     ┌───┐ ░ ├───┤│         │ ░  ║  ║  ║  ║ └╥┘┌─┐            »
           second_1: ┤ X ├─░─┤ X ├┤6  Adder ├─░──╫──╫──╫──╫──╫─┤M├────────────»
                     ├───┤ ░ ├───┤│         │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐         »
           second_2: ┤ X ├─░─┤ X ├┤7        ├─░──╫──╫──╫──╫──╫──╫─┤M├─────────»
                     └───┘ ░ ├───┤│         │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      »
           second_3: ──────░─┤ X ├┤8        ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├──────»
                           ░ └───┘│         │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   »
          ancilla_0: ──────░──────┤9        ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───»
                           ░      │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
          ancilla_1: ──────░──────┤10       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├»
                           ░      │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘»
          ancilla_2: ──────░──────┤11       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                           ░      │         │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
          ancilla_3: ──────░──────┤12       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
                           ░      └─────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
  first_classical: 4/════════════════════════════╩══╩══╩══╩══╬══╬══╬══╬══╬══╬═»
                                                 0  1  2  3  ║  ║  ║  ║  ║  ║ »
 second_classical: 4/════════════════════════════════════════╩══╩══╩══╩══╬══╬═»
                                                             0  1  2  3  ║  ║ »
ancilla_classical: 4/════════════════════════════════════════════════════╩══╩═»
                                                                         0  1 »
«                           
«              carry: ──────
«                           
«            first_0: ──────
«                           
«            first_1: ──────
«                           
«            first_2: ──────
«                           
«            first_3: ──────
«                           
«           second_0: ──────
«                           
«           second_1: ──────
«                           
«           second_2: ──────
«                           
«           second_3: ──────
«                           
«          ancilla_0: ──────
«                           
«          ancilla_1: ──────
«                     ┌─┐   
«          ancilla_2: ┤M├───
«                     └╥┘┌─┐
«          ancilla_3: ─╫─┤M├
«                      ║ └╥┘
«  first_classical: 4/═╬══╬═
«                      ║  ║ 
« second_classical: 4/═╬══╬═
«                      ║  ║ 
«ancilla_classical: 4/═╩══╩═
«                      2  3

In [8]:
job = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
processed_counts = resultToNumber(counts)
print("{} - {} ".format(firstNumber, secondNumber), processed_counts)

2 - 6  {'1100': 1000}


In [9]:
test_circuit = QuantumCircuit(4,4)
test_circuit.x(0)
test_circuit.measure([0,1], [0,1])
test_circuit.draw()

┌───┐┌─┐
q_0: ┤ X ├┤M├
     └┬─┬┘└╥┘
q_1: ─┤M├──╫─
      └╥┘  ║ 
q_2: ──╫───╫─
       ║   ║ 
q_3: ──╫───╫─
       ║   ║ 
c: 4/══╩═══╩═
       1   0

In [10]:
job = execute(test_circuit, Aer.get_backend('qasm_simulator'), shots=1000)
counts = job.result().get_counts()
print(counts)

{'0001': 1000}
